### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
from functools import reduce

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
merged_data_df = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

# Add columns with boolean values; 1 is for those that passed math and reading, else 0.

merged_data_df ['passed_math'] = np.where(merged_data_df["math_score"]>=70.0, 1, 0)

merged_data_df ['passed_reading'] = np.where(merged_data_df["reading_score"]>=70.0, 1, 0)

merged_data_df.head()

# Format all outputs to be used throughout the activitiy
percent_format="{0:,.2f}%".format
dollar_format="${0:,.2f}".format
number_format="{0:,}".format

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
# Determine the number of schools
ds_schools_total=len(school_data["school_name"].unique())

# Determine number of students at schools
ds_students_total=student_data["student_name"].count()

# Finding total budget
ds_budget_total=school_data["budget"].sum()

# The average math score
ds_average_mathscore = merged_data_df["math_score"].mean()

# The average reading score
ds_average_readingscore = merged_data_df["reading_score"].mean()

# The percentage of students with a passing reading score (70 or greater)
ds_pass_reading=merged_data_df[merged_data_df["reading_score"]>= 70.0].count()
ds_percent_pass_reading = ds_pass_reading["reading_score"]/ds_students_total*100

# The percentage of students with a passing math score (70 or greater)
ds_pass_math = merged_data_df[merged_data_df["math_score"]>= 70.0].count()
ds_percent_pass_math = ds_pass_math["math_score"]/ds_students_total*100

# Passing rate of averaged math and reading
ds_overall_passing_rate=(ds_average_mathscore + ds_average_readingscore)/2

# Dataframe for results and formatting
district_summary_df=pd.DataFrame([[ds_schools_total,
                                   ds_students_total,
                                   ds_budget_total,
                                   ds_average_mathscore,
                                   ds_average_readingscore,
                                   ds_percent_pass_math,
                                   ds_percent_pass_reading,
                                   ds_overall_passing_rate]],
                                  index=[0],
                                  columns = ["Total Schools",
                                         "Total Students",
                                         "Total Budget",
                                         "Average Math Score",
                                         "Average Reading Score",
                                         "% Passing Math",
                                         "% Passing Reading",
                                         "% Overall Passing Rate"]
                                 )

district_summary_df["Total Budget"]=district_summary_df["Total Budget"].map(dollar_format)
district_summary_df["Total Students"]=district_summary_df["Total Students"].map(number_format)
district_summary_df.iloc[:,3:8]=district_summary_df.iloc[:,3:8].applymap(percent_format)
district_summary_df.head()

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"39,170","$24,649,428.00",78.99%,81.88%,74.98%,85.81%,80.43%


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [35]:
# Dictionary of functions that need to included in the merged_df function to get school summary values:
    # Note that np (numberpy) is applied here due to us trying to find arithmetic functions
dict_aggregate = {'type':np.unique,
            'student_name':np.size,
            'budget':np.unique,
            'math_score':np.mean,
            'reading_score':np.mean,
            'passed_math': np.sum,
            'passed_reading': np.sum,
           }

columns = {'math_score': 'Average Math Score',
          'reading_score': 'Average Reading Score',
          'type': 'School Type',
          'student_name': 'Total Students',
          'passed_math': 'Total Stu Math Pass',
          'passed_reading': 'Total Stu Reading Pass',
          'budget': 'School Budget'
          } 
raw_dataset = merged_data_df.groupby(['school_name']).agg(dict_aggregate).rename(columns = columns)

# Identifying budget/student by school name
raw_dataset['Budget per Student']=raw_dataset['School Budget']/raw_dataset['Total Students']

# Identifying percent of students passing math (score of 70 or greater)/school name
raw_dataset['% Passing Math']=raw_dataset['Total Stu Math Pass']/raw_dataset['Total Students']*100

# Identifying percent of students passing reading (score of 70 or greater)/school name
raw_dataset['% Passing Reading']=raw_dataset['Total Stu Reading Pass']/raw_dataset['Total Students']*100

# The overall passing rate by school
raw_dataset['Overall Passing Rate']=(raw_dataset['% Passing Math'] + raw_dataset['% Passing Reading'])/2

# Columns for number of students passing math and reading
final_dataset=raw_dataset.drop(columns=['Total Stu Math Pass', 'Total Stu Reading Pass'])

# Ordering of columns
columns_datasets = final_dataset.columns.tolist()
order=[0,1,2,5,3,4,6,7,8]
columns_datasets=[ columns_datasets[i] for i in order]
final_dataset=final_dataset[columns_datasets]

# Formatting
final_dataset["School Budget"]=final_dataset["School Budget"].map(dollar_format)
final_dataset["Budget per Student"]=final_dataset["Budget per Student"].map(dollar_format)
final_dataset.iloc[:,4:9]=final_dataset.iloc[:,4:9].applymap(percent_format)

final_dataset.head()

,School Type,Total Students,School Budget,Budget per Student,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
school_name,,,,,,,,,
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.05%,81.03%,66.68%,81.93%,74.31%
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.06%,83.98%,94.13%,97.04%,95.59%
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.71%,81.16%,65.99%,80.74%,73.36%
Ford High School,District,2739,"$1,763,916.00",$644.00,77.10%,80.75%,68.31%,79.30%,73.80%
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.35%,83.82%,93.39%,97.14%,95.27%


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [17]:
# Ordering the Top 5
final_dataset_top = final_dataset.sort_values(by='Overall Passing Rate', ascending=False)
final_dataset_top.head()

,School Type,Total Students,School Budget,Budget per Student,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
school_name,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.06%,83.98%,94.13%,97.04%,95.59%
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.42%,83.85%,93.27%,97.31%,95.29%
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.35%,83.82%,93.39%,97.14%,95.27%
Pena High School,Charter,962,"$585,858.00",$609.00,83.84%,84.04%,94.59%,95.95%,95.27%
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.27%,83.99%,93.87%,96.54%,95.20%


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [18]:
# Ordering the Bottom 5
final_dataset_bottom = final_dataset.sort_values(by='Overall Passing Rate', ascending=True)
final_dataset_bottom.head()

,School Type,Total Students,School Budget,Budget per Student,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
school_name,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.84%,80.74%,66.37%,80.22%,73.29%
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.71%,81.16%,65.99%,80.74%,73.36%
Huang High School,District,2917,"$1,910,635.00",$655.00,76.63%,81.18%,65.68%,81.32%,73.50%
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.07%,80.97%,66.06%,81.22%,73.64%
Ford High School,District,2739,"$1,763,916.00",$644.00,77.10%,80.75%,68.31%,79.30%,73.80%


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [19]:
# Group school name and grade data set
grade_byschool = merged_data_df.groupby(['school_name', 'grade'])

# Average math score by school name and grade level
math_grade=grade_byschool['math_score'].mean()

# Take and make into a DataFrame
math_grade.to_frame()

# Turn the column values for grade level int0 column headers
math_grade_final=math_grade.unstack()

# rearrange the column headers to make "9th" grade come before "10th 11th 12th" grade
# put the column headers in a list 
cols=math_grade_final.columns.tolist()

# move the last column header in list to the first position in the list
cols=cols[-1:]+cols[:-1]

# reorder the dataframe as per column order set above
math_grade_final=math_grade_final[cols]

math_grade_final.head()

grade,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [38]:
# Reprsenting 9th grade
readingscore_9th=merged_data_df[merged_data_df['grade']=='9th'].groupby('school_name').mean()
reading9=readingscore_9th["reading_score"]
reading9=reading9.to_frame('9th Grade')

# Reprsenting 10th grade
readingscore_10th=merged_data_df[merged_data_df['grade']=='10th'].groupby('school_name').mean()
reading10=readingscore_10th["reading_score"]
reading10=reading10.to_frame('10th Grade')

# Reprsenting 11th grade
readingscore_11th=merged_data_df[merged_data_df['grade']=='11th'].groupby('school_name').mean()
reading11=readingscore_11th["reading_score"]
reading11=reading11.to_frame('11th Grade')

# Reprsenting 12th grade
readingscore_12th = merged_data_df[merged_data_df['grade']=='12th'].groupby('school_name').mean()
reading12=readingscore_12th["reading_score"]
reading12=reading12.to_frame('12th Grade')

# Setting up the dataframe for all grades
dfs=[reading9,reading10,reading11, reading12]

# Table representing all grades
Reading_Score_byGrade = reduce(lambda left,right: pd.merge(left,right,on='school_name'), dfs)
Reading_Score_byGrade.iloc[:,0:5]=Reading_Score_byGrade.iloc[:,0:5].applymap(percent_format)

Reading_Score_byGrade.head()

,9th Grade,10th Grade,11th Grade,12th Grade
school_name,,,,
Bailey High School,81.30%,80.91%,80.95%,80.91%
Cabrera High School,83.68%,84.25%,83.79%,84.29%
Figueroa High School,81.20%,81.41%,80.64%,81.38%
Ford High School,80.63%,81.26%,80.40%,80.66%
Griffin High School,83.37%,83.71%,84.29%,84.01%


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [21]:
# Identifying the min and max for the bins
print(final_dataset["Budget per Student"].min())
print(final_dataset["Budget per Student"].max())

$578.00
$655.00


In [44]:
# Sample bins
spending_bins = [0, 600, 625, 655, 675]
group_names = ["<$600", "$600-625", "$625-655", "$655-675"]

# Design columns based on bins
raw_dataset["Spending Ranges (Per Student)"]=pd.cut(raw_dataset["Budget per Student"], spending_bins, labels=group_names)

# Design new dictionary
dict_aggregate_2 = {'Average Math Score':np.mean,
            'Average Reading Score':np.mean,
            'Total Stu Math Pass':np.sum,
            'Total Stu Reading Pass':np.sum,
             'Total Students':np.sum
             }

# Categorize spending based on size and apply calculations for the dictionary
SS_raw = raw_dataset.groupby(['Spending Ranges (Per Student)']).agg(dict_aggregate_2)

# For passing scores
SS_raw["%Passing Math"]=SS_raw["Total Stu Math Pass"]/SS_raw["Total Students"]*100
SS_raw["%Passing Reading"]=SS_raw["Total Stu Reading Pass"]/SS_raw["Total Students"]*100
SS_raw["Overall Passing"]=(SS_raw["%Passing Reading"]+SS_raw["%Passing Math"])/2

# Remove columns from final dataframe
SS_final=SS_raw.drop(columns=['Total Stu Math Pass', 'Total Stu Reading Pass', 'Total Students'])

# Final table format
SS_final.iloc[:,0:5]=SS_final.iloc[:,0:5].applymap(percent_format)

SS_final.head()

,Average Math Score,Average Reading Score,%Passing Math,%Passing Reading,Overall Passing
Spending Ranges (Per Student),,,,,
<$600,83.44%,83.89%,93.74%,96.51%,95.12%
$600-625,83.60%,83.93%,93.87%,96.67%,95.27%
$625-655,77.76%,81.33%,68.05%,81.84%,74.94%
$655-675,nan%,nan%,nan%,nan%,nan%


## Scores by School Size

* Perform the same operations as above, based on school size.

In [23]:
# Sample bins.
size_bins = [0, 1000, 3000, 5000]
group_names2 = ["Small (<1000)", "Medium (1000-3000)", "Large (3000-5000)"]

# Columsn for bins
raw_dataset["School Size"]=pd.cut(raw_dataset["Total Students"], size_bins, labels=group_names2)

# Apply functions in from aggregated dictionary for school sizes
schoolsize_original = raw_dataset.groupby(['School Size']).agg(dict_aggregate_2)

# Calculations
schoolsize_original["%Passing Math"]=schoolsize_original["Total Stu Math Pass"]/schoolsize_original["Total Students"]*100
schoolsize_original["%Passing Reading"]=schoolsize_original["Total Stu Reading Pass"]/schoolsize_original["Total Students"]*100
schoolsize_original["Overall Passing"]=(schoolsize_original["%Passing Reading"]+schoolsize_original["%Passing Math"])/2

# Final dataframe
schoolsize_original_final=schoolsize_original.drop(columns=['Total Stu Math Pass', 'Total Stu Reading Pass', 'Total Students'])

# Format the table
schoolsize_original_final.iloc[:,0:5]=schoolsize_original_final.iloc[:,0:5].applymap(percent_format)

schoolsize_original_final.head()

,Average Math Score,Average Reading Score,%Passing Math,%Passing Reading,Overall Passing
School Size,,,,,
Small (<1000),83.82%,83.93%,93.95%,96.04%,95.00%
Medium (1000-3000),81.18%,82.93%,81.68%,89.52%,85.60%
Large (3000-5000),77.06%,80.92%,66.47%,81.11%,73.79%


## Scores by School Type

* Perform the same operations as above, based on school type

In [43]:
# Grouping by school type category
school_type_raw = raw_dataset.groupby(['School Type']).agg(dict_aggregate_2)

# Calculations
school_type_raw["%Passing Math"]=school_type_raw["Total Stu Math Pass"]/school_type_raw["Total Students"]*100
school_type_raw["%Passing Reading"]=school_type_raw["Total Stu Reading Pass"]/school_type_raw["Total Students"]*100
school_type_raw["Overall Passing"]=(school_type_raw["%Passing Reading"]+school_type_raw["%Passing Math"])/2

# Final dataframe
school_type_final=school_type_raw.drop(columns=['Total Stu Math Pass', 'Total Stu Reading Pass', 'Total Students'])

# Format the table
school_type_final.iloc[:,0:5]=school_type_final.iloc[:,0:5].applymap(percent_format)

school_type_final.head()

,Average Math Score,Average Reading Score,%Passing Math,%Passing Reading,Overall Passing
School Type,,,,,
Charter,83.47%,83.90%,93.70%,96.65%,95.17%
District,76.96%,80.97%,66.52%,80.91%,73.71%
